In [5]:
import pandas as pd
import requests
import time
import os

In [7]:
# Caminhos dos arquivos
BASICS_PATH = 'title.basics.tsv'
RATINGS_PATH = 'title.ratings.tsv'
OUTPUT_PATH = 'meu_dataset_completo.csv'
API_KEY = '37f9c1da'

# 1. Carregar os dados IMDb
basics = pd.read_csv(BASICS_PATH, sep='\t', low_memory=False)
ratings = pd.read_csv(RATINGS_PATH, sep='\t')

# 2. Filtrar apenas filmes e colunas úteis
movies = basics[basics['titleType'] == 'movie']
movies = movies[['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'genres']]

# 3. Unir com os ratings
movies = movies.merge(ratings, on='tconst')

# 4. Enriquecer com dados da OMDb API
url_base = "http://www.omdbapi.com/?apikey={}&i={}"
dados_completos = []

for i, row in movies.head(100).iterrows():
    imdb_id = row['tconst']
    response = requests.get(url_base.format(API_KEY, imdb_id))
    if response.status_code == 200:
        data = response.json()
        if data['Response'] == 'True':
            dados_completos.append({
                'imdb_id': imdb_id,
                'title': data.get('Title'),
                'year': data.get('Year'),
                'genre': data.get('Genre'),
                'rating': data.get('imdbRating'),
                'plot': data.get('Plot'),
                'poster': data.get('Poster')
            })
    time.sleep(0.3)  # Para evitar bloqueios

# 5. Criar DataFrame final
final_df = pd.DataFrame(dados_completos)
final_df.to_csv(OUTPUT_PATH, index=False)
print("Dataset salvo em:", OUTPUT_PATH)


Dataset salvo em: meu_dataset_completo.csv
